#### 1. data set preprocessing
* 원본 데이터셋은 class가 뒤죽박죽 섞여 있었음.
* 이를 class별로 정리하기 위한 전처리
    * air_conditioner, car_horn, children_playing, dog_bark, drilling
    * engine_idllling, gun_shot, jack_hammer, siren, street_music

In [ ]:
import glob
import shutil

arr = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Name = ["air_conditioner", "car_horn", "children_playing", "dog_bark",
       "drilling", "engine_idlling", "gun_shot", "jackhammer", "siren", "street_music"]

def name(num):
    arr[num] = arr[num] + 1
    return Name[num] + str(arr[num])

In [ ]:
for i in range(1, 11):
    folderName = "fold"+str(i)
    fileList = glob.glob("1_origindata/"+folderName+"/*.wav")
    for j in range(len(fileList)):
        tempSplit = fileList[j].split("-")
        originfileName = fileList[j]
        newfileName = "2_data/fold"+str(tempSplit[1])+"/"+name(int(tempSplit[1]))+".wav"
        shutil.copy(originfileName, newfileName)
        print("Success copy", end = " : ")
        print(originfileName + " -> " + newfileName)
    print("/// " + folderName + " finish..")
print("all finish...")

---

### 2. convert wav to jpg
* librosa 라이브러리르 이용하여 wav파일을 jpg파일로 변환

* 시각화 결과 0, 2, 5, 7, 9번 class는 무의미하고 판별이 어려울 것으로 판단
* 1, 3, 4, 6, 8번 class로 classifier 개발 시작. + 위의 5개 class는 배경 백색 소음 역할 수행

* 즉, 디렉토리는 0 to 9가 아닌, 0 to 4로 재 정렬된다.
* fold0 : car_horn
* fold1 : dog_bark
* fold2 : drilling
* fold3 : gun_shot
* fold4 : siren

* 단, 학습 완료 후, 프로젝트 진행이 안정화되었을 때, 10개 class판별을 위한 upgrade 진행을 할 수 있으며, 이를 위한 origin data는 보관한다.

In [ ]:
import librosa
import librosa.display
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.font_manager as fm
import glob
%matplotlib inline

In [ ]:
for i in range(10):
    folderName = "fold" + str(i)
    print("======"+folderName+"======")
    fileList = glob.glob("2_data/"+folderName+"/*.wav")
    for j in range(len(fileList)):
        tempSplit = fileList[j].split(".")
        tempSplit2 = tempSplit[0].split("\\")
        fileName = tempSplit2[-1]
        
        plt.figure()
        y, sr = librosa.load(fileList[j])
        S = np.abs(librosa.stft(y))
        librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                                 sr = sr, y_axis = 'log', x_axis = 'time')
        #plt.colorbar(format = '%+2.0f dB')
        plt.title('Log-Power spectrogram')
        #plt.show()
        print("Save file Complete : 3_image/"+folderName+"/"+fileName+".jpg")
        plt.savefig("3_image/"+folderName+"/"+fileName+".jpg")
        plt.close()

In [ ]:
plt.figure()
y, sr = librosa.load("파일경로")
S = np.abs(librosa.stft(y))
librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                         sr = sr, y_axis = 'log', x_axis = 'time')
plt.show()
plt.close()

In [ ]:
# 시각화 origin code backup
"""
plt.figure()
y, sr = librosa.load(fileList[0])
S = np.abs(librosa.stft(y))
librosa.display.specshow(librosa.power_to_db(S**2, ref = np.max),
                         sr = sr, y_axis = 'log', x_axis = 'time')
plt.colorbar(format = '%+2.0f dB')
plt.title('Log-Power spectrogram')
plt.show()
"""

---

### 3. jpg 사진 자르기
* 필요한 부분만 자르기
* 여기서 fold2는 배경 class로 선정, 즉 fold5에 다시 저장한다

* 초기에는 5개 폴더를 모두 배경 잡음으로 선택하였다. 이때, 모든 데이터를 저장하면 5000장으로 다른 데이터셋에 비해 과하게 많은 양을 차지하므로 1/5로 축소할 필요가 있었으나, 지금은 필요없다

In [ ]:
from PIL import Image
from tqdm import tqdm_notebook
import glob
"""
img = Image.open("test.jpg")
area = (54, 32, 385, 253)
cropped_img = img.crop(area)
cropped_img.save("crop.jpg")
"""

In [ ]:
for i in tqdm_notebook(range(6)):
    folderName = "fold" + str(i)
    print("======"+folderName+"======")
    fileList = glob.glob("3_image/"+folderName+"/*.jpg")
    for j in range(len(fileList)):
        fileName = fileList[j].split("\\")[1]
        img = Image.open(fileList[j])
        area = (54, 32, 385, 253)
        cropped_img = img.crop(area)
        cropped_img.save("4_imageData/"+folderName+"/"+fileName)
        print("Image Split complete : " + fileName)

---

### 4. split dataset
* 통합 database를 train, validation, test로 분할
* 전체 데이터 10이라 할때, train(6), val(2), test(2)

In [ ]:
import glob
import shutil

for i in range(6):
    if(i!=5):
        continue
    folderName = "fold" + str(i)
    fileList = glob.glob("4_imageData/"+folderName+"/*.jpg")
    trainRange = int(len(fileList)*0.6)
    validationRange = int(len(fileList)*0.2)
    
    print("train : 0:"+str(trainRange))
    print("validation : "+str(trainRange+1)+":"+str(trainRange+validationRange))
    print("test : "+str(trainRange+validationRange+1)+":"+str(len(fileList)))
    
    # train data
    for j in range(trainRange+1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]
        
        newfileName = "5_dataset/train/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)

    # validation data
    for j in range(trainRange+1, trainRange + validationRange + 1):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "5_dataset/val/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)
        
    # test data
    for j in range(trainRange + validationRange + 1, len(fileList)):
        originfileName = fileList[j]
        tempSplit = fileList[j].split("/")
        fileName = tempSplit[-1]        
        
        newfileName = "5_dataset/test/"+fileName
        shutil.copy(originfileName, newfileName)
        print("copy succes : " + originfileName + " -> " + newfileName)        

In [ ]:
import glob

print("class 별 data set 개수 측정")
print("========== car_horn ==========")
print('train car_horn image count : ' + str(len(glob.glob("5_dataset/train/fold0/*jpg"))))
print('validation car_horn image count : ' + str(len(glob.glob("5_dataset/val/fold0/*jpg"))))
print('test car_horn image count : ' + str(len(glob.glob("5_dataset/test/fold0/*jpg"))))

print("========== dog_bark ==========")
print('train dog_bark image count : ' + str(len(glob.glob("5_dataset/train/fold1/*jpg"))))
print('validation dog_bark image count : ' + str(len(glob.glob("5_dataset/val/fold1/*jpg"))))
print('test dog_bark image count : ' + str(len(glob.glob("5_dataset/test/fold1/*jpg"))))

print("========== drilling ==========")
print('train drilling image count : ' + str(len(glob.glob("5_dataset/train/fold2/*jpg"))))
print('validation drilling image count : ' + str(len(glob.glob("5_dataset/val/fold2/*jpg"))))
print('test drilling image count : ' + str(len(glob.glob("5_dataset/test/fold2/*jpg"))))

print("========== gun_shot ==========")
print('train gun_shot image count : ' + str(len(glob.glob("5_dataset/train/fold3/*jpg"))))
print('validation gun_shot image count : ' + str(len(glob.glob("5_dataset/val/fold3/*jpg"))))
print('test gun_shot image count : ' + str(len(glob.glob("5_dataset/test/fold3/*jpg"))))

print("========== siren ==========")
print('train siren image count : ' + str(len(glob.glob("5_dataset/train/fold4/*jpg"))))
print('validation siren image count : ' + str(len(glob.glob("5_dataset/val/fold4/*jpg"))))
print('test siren image count : ' + str(len(glob.glob("5_dataset/test/fold4/*jpg"))))

print("========== 배경음 ==========")
print('train background image count : ' + str(len(glob.glob("5_dataset/train/fold5/*jpg"))))
print('validation background image count : ' + str(len(glob.glob("5_dataset/val/fold5/*jpg"))))
print('test background image count : ' + str(len(glob.glob("5_dataset/test/fold5/*jpg"))))